In [1]:
import os
import numpy as np
from scipy.fftpack import fft
from scipy.io import wavfile
from scipy import signal
from glob import glob
import re
import pandas as pd
import gc
from scipy.io import wavfile

from keras.models import load_model
from keras.utils.np_utils import to_categorical
from keras import optimizers, losses, activations, models
from keras.utils.vis_utils import plot_model
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization
from sklearn.model_selection import train_test_split
import keras

Using TensorFlow backend.


In [2]:
# new_sample_rate = 8000

LABELS = ['_silence', '_unknown', 'down', 'go', 'left', 'no', 'off', 'on', 'right', 'stop', 'up', 'yes']
TRAIN_PATH = './input/train/audio/'
TEST_PATH = './input/test/audio/'
OUTPUT_PATH = './output/'
MODEL_NAME = 'cnn_new.h5'

# class_weight = {0: 13.0,
#  1: 1.0,
#  2: 9.0,
#  3: 9.0,
#  4: 9.0,
#  5: 9.0,
#  6: 9.0,
#  7: 5.0,
#  8: 9.0,
#  9: 9.0,
#  10: 9.0,
#  11: 9.0}

# class_weight = {0: 10.0,
#  1: 1.0,
#  2: 1.0,
#  3: 1.0,
#  4: 1.0,
#  5: 1.0,
#  6: 1.0,
#  7: 1.0,
#  8: 1.0,
#  9: 1.0,
#  10: 1.0,
#  11: 1.0}

In [3]:
# 원본 클래스 비율
# {0: 1200,
#  1: 41039,
#  2: 2359,
#  3: 2372,
#  4: 2353,
#  5: 2375,
#  6: 2357,
#  7: 2367,
#  8: 2367,
#  9: 2380,
#  10: 2375,
#  11: 2377}

In [4]:
## custom_fft and log_specgram functions written by DavidS.
def custom_fft(y, fs):
    T = 1.0 / fs
    N = y.shape[0]
    yf = fft(y)
    xf = np.linspace(0.0, 1.0/(2.0*T), N//2)
    # FFT 는 대칭(simmetrical)이므로 반쪽만 얻음.
    # FFT 는 복소수이므로 실수값만 취하기 위해 abs()
    vals = 2.0/N * np.abs(yf[0:N//2])
    return xf, vals

def log_specgram(audio, sample_rate, window_size=20, step_size=10, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    freqs, times, spec = signal.spectrogram(audio,
                                    fs=sample_rate,
                                    window='hann',
                                    nperseg=nperseg,
                                    noverlap=noverlap,
                                    detrend=False)
    return freqs, times, np.log(spec.T.astype(np.float32) + eps)

In [5]:
## utility function to grab all wav files inside train data folder.
def list_wavs_fname(dirpath, ext='wav'):
    print(dirpath)
    fpaths = glob(os.path.join(dirpath, r'*/*' + ext))
    pat = r'.+/(\w+)/\w+\.' + ext + '$'
    labels = []
    for fpath in fpaths:
        r = re.match(pat, fpath)
        if r:
            labels.append(r.group(1))
    pat = r'.+/(\w+\.' + ext + ')$'
    fnames = []
    for fpath in fpaths:
        r = re.match(pat, fpath)
        if r:
            fnames.append(r.group(1))
    return labels, fnames

In [6]:
def pad_audio(samples, L=16000):
    '''
    pad audios that are less than 16000(1 second) with 0s to make them all have the same length.
    '''
    if len(samples) >= L: 
        return samples
    else: 
        return np.pad(samples, pad_width=(L - len(samples), 0), mode='constant', constant_values=(0, 0)) 
        # sample 앞뒤로 constant_values[0]과 constant_values[1]을 각각 pad_width 갯수 만큼 패딩
        # 총길이는 len(samples) + 2*pad_width

# def chop_audio(samples, L=16000, num=200):
#     '''
#     chop audios that are larger than 16000(eg. wav files in background noises folder) to 16000 in length.
#     create several chunks out of one large wav files given the parameter 'num'.
#     '''
#     for i in range(num):
#         beg = np.random.randint(0, len(samples) - L)
#         yield samples[beg: beg + L]

def label_transform(labels):
    '''
    레이블 정규화 및 one-hot벡터화 (더미화)
    '''
    nlabels = []
    for label in labels:
        if label == '_background_noise_':
            nlabels.append('_silence')
        elif label not in LABELS:
            nlabels.append('_unknown')
        else:
            nlabels.append(label)
    encoder = LabelEncoder()
    encoder.fit(nlabels)
    nlabels = encoder.transform(nlabels)
    return nlabels

# 1. Load Data & Preprocessing

### 1) Load labels, fnames

In [7]:
labels, fnames = list_wavs_fname(TRAIN_PATH)

./input/train/audio/


### 2) Feature Extraction

In [8]:
# sample_rate, samples = wavfile.read(os.path.join(TRAIN_PATH, labels[0], fnames[0]))
# len(samples)

In [9]:
%%time
y = []
X = []
for i, (label, fname) in enumerate(zip(labels, fnames)):
    sample_rate, samples = wavfile.read(os.path.join(TRAIN_PATH, label, fname))
    if len(samples) > 16000:
        pass
    else:
        samples = pad_audio(samples)
#         resampled = signal.resample(samples, int(new_sample_rate / sample_rate * samples.shape[0]))
#         _, _, specgram = log_specgram(resampled, sample_rate=new_sample_rate)
        _, _, specgram = log_specgram(samples, sample_rate=16000)
        y.append(label)
        X.append(specgram)

CPU times: user 1min 23s, sys: 3.18 s, total: 1min 26s
Wall time: 1min 26s


/home/chi/anaconda3/lib/python3.5/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [10]:
X = np.array(X)
X = X.reshape(tuple(list(X.shape) + [1])) # (64841, 99, 81, 1) 로 reshape
y = to_categorical(label_transform(y))

### 3) Train Validation Set Split

In [11]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.05, random_state=1130) # 9:1로 train, valid 셋 나눔.
del X, y
gc.collect()

0

In [12]:
# model = load_model(os.path.join(OUTPUT_PATH, MODEL_NAME))

# 2. Modeling

In [12]:
input_shape = (99, 161, 1) #(99, 81, 1) # in order to fit into Conv2D layer, we need to reshape it.
nclass = 12

In [20]:
## Modeling
inp = Input(shape=input_shape)
norm_inp = BatchNormalization()(inp)
img_1 = Convolution2D(8, kernel_size=2, activation=activations.relu)(norm_inp)
img_1 = Convolution2D(8, kernel_size=2, activation=activations.relu)(img_1)
img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
img_1 = Dropout(rate=0.2)(img_1)
img_1 = Convolution2D(16, kernel_size=3, activation=activations.relu)(img_1)
img_1 = Convolution2D(16, kernel_size=3, activation=activations.relu)(img_1)
img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
img_1 = Convolution2D(32, kernel_size=3, activation=activations.relu)(img_1)
img_1 = Convolution2D(32, kernel_size=3, activation=activations.relu)(img_1)
img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
img_1 = Dropout(rate=0.2)(img_1)
img_1 = Convolution2D(32, kernel_size=3, activation=activations.relu)(img_1)
img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
img_1 = Dropout(rate=0.2)(img_1)
img_1 = Flatten()(img_1)

dense_1 = BatchNormalization()(Dense(128, activation=activations.relu)(img_1))
dense_1 = Dropout(rate=0.2)(dense_1)
dense_1 = BatchNormalization()(Dense(128, activation=activations.relu)(dense_1))
dense_1 = Dense(128, activation=activations.relu)(dense_1)
dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

model = models.Model(inputs=inp, outputs=dense_1)
opt = optimizers.Adam()

model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=['categorical_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 99, 161, 1)        0         
_________________________________________________________________
batch_normalization_16 (Batc (None, 99, 161, 1)        4         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 98, 160, 8)        40        
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 97, 159, 8)        264       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 48, 79, 8)         0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 48, 79, 8)         0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 46, 77, 16)        1168      
__________

In [21]:
# ## Modeling
# inp = Input(shape=input_shape)
# norm_inp = BatchNormalization()(inp)
# img_1 = Convolution2D(8, kernel_size=2, activation=activations.relu, padding='same')(norm_inp)
# img_1 = Convolution2D(8, kernel_size=2, activation=activations.relu, padding='same')(img_1)
# img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
# img_1 = Dropout(rate=0.2)(img_1)
# img_1 = Convolution2D(16, kernel_size=2, activation=activations.relu, padding='same')(img_1)
# img_1 = Convolution2D(16, kernel_size=2, activation=activations.relu, padding='same')(img_1)
# img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
# img_1 = Dropout(rate=0.2)(img_1)
# img_1 = Convolution2D(16, kernel_size=3, activation=activations.relu, padding='same')(img_1)
# img_1 = Convolution2D(16, kernel_size=3, activation=activations.relu, padding='same')(img_1)
# img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
# img_1 = Dropout(rate=0.2)(img_1)
# img_1 = Convolution2D(32, kernel_size=3, activation=activations.relu, padding='same')(img_1)
# img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
# img_1 = Dropout(rate=0.2)(img_1)
# img_1 = Flatten()(img_1)

# dense_1 = BatchNormalization()(img_1)
# dense_1 = Dense(512, activation=activations.relu)(dense_1)
# dense_1 = Dropout(rate=0.2)(dense_1)
# dense_1 = Dense(512, activation=activations.relu)(dense_1)
# dense_1 = Dropout(rate=0.2)(dense_1)
# dense_1 = Dense(512, activation=activations.relu)(dense_1)
# dense_1 = Dropout(rate=0.2)(dense_1)
# dense_1 = Dense(128, activation=activations.relu)(dense_1)
# dense_1 = Dropout(rate=0.2)(dense_1)
# dense_1 = Dense(64, activation=activations.relu)(dense_1)
# dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

# model = models.Model(inputs=inp, outputs=dense_1)
# opt = optimizers.Adam(lr=0.001)

# model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=['categorical_accuracy'])
# model.summary()

In [22]:
# 네트워크 시각화
# plot_model(model, to_file='output/model_plot.png', show_shapes=True, show_layer_names=True)

In [23]:
%%time
# model.fit(X_train, y_train, batch_size=2048, validation_data=(X_valid, y_valid), \
#           class_weight=class_weight, epochs=300, shuffle=True, verbose=1)
model.fit(X_train, y_train, batch_size=1024, validation_data=(X_valid, y_valid), \
          epochs=200, shuffle=True, verbose=1)
model.save(os.path.join(OUTPUT_PATH, MODEL_NAME))

Train on 93413 samples, validate on 4917 samples
Epoch 1/200
93413/93413 [==============================] - 49s - loss: 0.2056 - categorical_accuracy: 0.5676 - val_loss: 0.2153 - val_categorical_accuracy: 0.6179
Epoch 2/200
93413/93413 [==============================] - 48s - loss: 0.1693 - categorical_accuracy: 0.6424 - val_loss: 0.1883 - val_categorical_accuracy: 0.6179
Epoch 3/200
93413/93413 [==============================] - 48s - loss: 0.1256 - categorical_accuracy: 0.7044 - val_loss: 0.1899 - val_categorical_accuracy: 0.6179
Epoch 4/200
93413/93413 [==============================] - 48s - loss: 0.0915 - categorical_accuracy: 0.7807 - val_loss: 0.1513 - val_categorical_accuracy: 0.6347
Epoch 5/200
93413/93413 [==============================] - 48s - loss: 0.0754 - categorical_accuracy: 0.8212 - val_loss: 0.0986 - val_categorical_accuracy: 0.7393
Epoch 6/200
93413/93413 [==============================] - 49s - loss: 0.0647 - categorical_accuracy: 0.8489 - val_loss: 0.0670 - val_ca

93413/93413 [==============================] - 48s - loss: 0.0193 - categorical_accuracy: 0.9569 - val_loss: 0.0154 - val_categorical_accuracy: 0.9656
Epoch 51/200
93413/93413 [==============================] - 48s - loss: 0.0190 - categorical_accuracy: 0.9571 - val_loss: 0.0157 - val_categorical_accuracy: 0.9642
Epoch 52/200
93413/93413 [==============================] - 49s - loss: 0.0189 - categorical_accuracy: 0.9575 - val_loss: 0.0153 - val_categorical_accuracy: 0.9660
Epoch 53/200
93413/93413 [==============================] - 49s - loss: 0.0184 - categorical_accuracy: 0.9586 - val_loss: 0.0153 - val_categorical_accuracy: 0.9650
Epoch 54/200
93413/93413 [==============================] - 49s - loss: 0.0183 - categorical_accuracy: 0.9598 - val_loss: 0.0152 - val_categorical_accuracy: 0.9650
Epoch 55/200
93413/93413 [==============================] - 48s - loss: 0.0185 - categorical_accuracy: 0.9585 - val_loss: 0.0169 - val_categorical_accuracy: 0.9614
Epoch 56/200
93413/93413 [===

93413/93413 [==============================] - 48s - loss: 0.0140 - categorical_accuracy: 0.9688 - val_loss: 0.0115 - val_categorical_accuracy: 0.9754
Epoch 101/200
93413/93413 [==============================] - 48s - loss: 0.0138 - categorical_accuracy: 0.9694 - val_loss: 0.0131 - val_categorical_accuracy: 0.9719
Epoch 102/200
93413/93413 [==============================] - 48s - loss: 0.0137 - categorical_accuracy: 0.9695 - val_loss: 0.0125 - val_categorical_accuracy: 0.9730
Epoch 103/200
93413/93413 [==============================] - 48s - loss: 0.0140 - categorical_accuracy: 0.9682 - val_loss: 0.0117 - val_categorical_accuracy: 0.9750
Epoch 104/200
93413/93413 [==============================] - 48s - loss: 0.0135 - categorical_accuracy: 0.9695 - val_loss: 0.0115 - val_categorical_accuracy: 0.9764
Epoch 105/200
93413/93413 [==============================] - 48s - loss: 0.0134 - categorical_accuracy: 0.9699 - val_loss: 0.0116 - val_categorical_accuracy: 0.9738
Epoch 106/200
93413/9341

93413/93413 [==============================] - 48s - loss: 0.0118 - categorical_accuracy: 0.9740 - val_loss: 0.0107 - val_categorical_accuracy: 0.9758
Epoch 150/200
93413/93413 [==============================] - 48s - loss: 0.0115 - categorical_accuracy: 0.9741 - val_loss: 0.0111 - val_categorical_accuracy: 0.9756
Epoch 151/200
93413/93413 [==============================] - 48s - loss: 0.0117 - categorical_accuracy: 0.9739 - val_loss: 0.0117 - val_categorical_accuracy: 0.9750
Epoch 152/200
93413/93413 [==============================] - 48s - loss: 0.0117 - categorical_accuracy: 0.9740 - val_loss: 0.0123 - val_categorical_accuracy: 0.9758
Epoch 153/200
93413/93413 [==============================] - 48s - loss: 0.0116 - categorical_accuracy: 0.9742 - val_loss: 0.0100 - val_categorical_accuracy: 0.9780
Epoch 154/200
93413/93413 [==============================] - 48s - loss: 0.0112 - categorical_accuracy: 0.9745 - val_loss: 0.0101 - val_categorical_accuracy: 0.9770
Epoch 155/200
93413/9341

93413/93413 [==============================] - 48s - loss: 0.0104 - categorical_accuracy: 0.9771 - val_loss: 0.0091 - val_categorical_accuracy: 0.9803
Epoch 199/200
93413/93413 [==============================] - 48s - loss: 0.0101 - categorical_accuracy: 0.9777 - val_loss: 0.0103 - val_categorical_accuracy: 0.9797
Epoch 200/200
93413/93413 [==============================] - 48s - loss: 0.0102 - categorical_accuracy: 0.9771 - val_loss: 0.0093 - val_categorical_accuracy: 0.9793
CPU times: user 1h 50min 12s, sys: 17min 40s, total: 2h 7min 52s
Wall time: 2h 42min 51s


In [24]:
model.fit(X_train, y_train, batch_size=1024, validation_data=(X_valid, y_valid), \
          epochs=200, shuffle=True, verbose=1)
model.save(os.path.join(OUTPUT_PATH, 'cnn_new_400.h5'))

Train on 93413 samples, validate on 4917 samples
Epoch 1/200
93413/93413 [==============================] - 48s - loss: 0.0100 - categorical_accuracy: 0.9772 - val_loss: 0.0101 - val_categorical_accuracy: 0.9793
Epoch 2/200
93413/93413 [==============================] - 48s - loss: 0.0102 - categorical_accuracy: 0.9775 - val_loss: 0.0100 - val_categorical_accuracy: 0.9793
Epoch 3/200
93413/93413 [==============================] - 48s - loss: 0.0103 - categorical_accuracy: 0.9773 - val_loss: 0.0103 - val_categorical_accuracy: 0.9782
Epoch 4/200
93413/93413 [==============================] - 48s - loss: 0.0104 - categorical_accuracy: 0.9771 - val_loss: 0.0101 - val_categorical_accuracy: 0.9776
Epoch 5/200
93413/93413 [==============================] - 48s - loss: 0.0096 - categorical_accuracy: 0.9785 - val_loss: 0.0099 - val_categorical_accuracy: 0.9788
Epoch 6/200
93413/93413 [==============================] - 48s - loss: 0.0100 - categorical_accuracy: 0.9779 - val_loss: 0.0100 - val_ca

93413/93413 [==============================] - 48s - loss: 0.0094 - categorical_accuracy: 0.9793 - val_loss: 0.0100 - val_categorical_accuracy: 0.9801
Epoch 51/200
93413/93413 [==============================] - 48s - loss: 0.0095 - categorical_accuracy: 0.9791 - val_loss: 0.0097 - val_categorical_accuracy: 0.9801
Epoch 52/200
93413/93413 [==============================] - 48s - loss: 0.0093 - categorical_accuracy: 0.9797 - val_loss: 0.0095 - val_categorical_accuracy: 0.9797
Epoch 53/200
93413/93413 [==============================] - 48s - loss: 0.0096 - categorical_accuracy: 0.9788 - val_loss: 0.0091 - val_categorical_accuracy: 0.9821
Epoch 54/200
93413/93413 [==============================] - 48s - loss: 0.0093 - categorical_accuracy: 0.9792 - val_loss: 0.0087 - val_categorical_accuracy: 0.9813
Epoch 55/200
93413/93413 [==============================] - 48s - loss: 0.0092 - categorical_accuracy: 0.9796 - val_loss: 0.0098 - val_categorical_accuracy: 0.9805
Epoch 56/200
93413/93413 [===

93413/93413 [==============================] - 48s - loss: 0.0086 - categorical_accuracy: 0.9809 - val_loss: 0.0096 - val_categorical_accuracy: 0.9815
Epoch 101/200
93413/93413 [==============================] - 48s - loss: 0.0087 - categorical_accuracy: 0.9809 - val_loss: 0.0096 - val_categorical_accuracy: 0.9805
Epoch 102/200
93413/93413 [==============================] - 48s - loss: 0.0084 - categorical_accuracy: 0.9815 - val_loss: 0.0095 - val_categorical_accuracy: 0.9823
Epoch 103/200
93413/93413 [==============================] - 48s - loss: 0.0089 - categorical_accuracy: 0.9805 - val_loss: 0.0090 - val_categorical_accuracy: 0.9813
Epoch 104/200
93413/93413 [==============================] - 48s - loss: 0.0086 - categorical_accuracy: 0.9813 - val_loss: 0.0094 - val_categorical_accuracy: 0.9815
Epoch 105/200
93413/93413 [==============================] - 48s - loss: 0.0086 - categorical_accuracy: 0.9807 - val_loss: 0.0098 - val_categorical_accuracy: 0.9799
Epoch 106/200
93413/9341

93413/93413 [==============================] - 48s - loss: 0.0083 - categorical_accuracy: 0.9814 - val_loss: 0.0091 - val_categorical_accuracy: 0.9819
Epoch 150/200
93413/93413 [==============================] - 48s - loss: 0.0080 - categorical_accuracy: 0.9825 - val_loss: 0.0093 - val_categorical_accuracy: 0.9823
Epoch 151/200
93413/93413 [==============================] - 48s - loss: 0.0081 - categorical_accuracy: 0.9821 - val_loss: 0.0088 - val_categorical_accuracy: 0.9813
Epoch 152/200
93413/93413 [==============================] - 48s - loss: 0.0081 - categorical_accuracy: 0.9815 - val_loss: 0.0084 - val_categorical_accuracy: 0.9823
Epoch 153/200
93413/93413 [==============================] - 48s - loss: 0.0079 - categorical_accuracy: 0.9824 - val_loss: 0.0093 - val_categorical_accuracy: 0.9817
Epoch 154/200
93413/93413 [==============================] - 48s - loss: 0.0079 - categorical_accuracy: 0.9822 - val_loss: 0.0087 - val_categorical_accuracy: 0.9813
Epoch 155/200
93413/9341

93413/93413 [==============================] - 48s - loss: 0.0076 - categorical_accuracy: 0.9830 - val_loss: 0.0088 - val_categorical_accuracy: 0.9833
Epoch 199/200
93413/93413 [==============================] - 48s - loss: 0.0080 - categorical_accuracy: 0.9825 - val_loss: 0.0098 - val_categorical_accuracy: 0.9799
Epoch 200/200
93413/93413 [==============================] - 48s - loss: 0.0078 - categorical_accuracy: 0.9829 - val_loss: 0.0085 - val_categorical_accuracy: 0.9831


# 3. Validation

In [25]:
preds_proba = model.predict(X_valid, batch_size=1024, verbose=1)

4917/4917 [==============================] - 1s     


In [26]:
preds = [LABELS[i] for i in np.argmax(preds_proba, axis=1)]
actuals = [LABELS[i] for i in np.argmax(y_valid, axis=1)]
print('* 정확도 : %.5f' % (np.sum(np.array(actuals) == np.array(preds)) / float(len(actuals))))
preds = pd.Categorical(preds, categories=LABELS)
actuals = pd.Categorical(actuals, categories=LABELS)
print('\n -------------------------- \n')
print(pd.crosstab(actuals, preds, rownames=['actuals'], colnames=['preds']))
print('\n -------------------------- \n')
print(classification_report(actuals, preds))

* 정확도 : 0.98312

 -------------------------- 

preds     _silence  _unknown  down   go  left   no  off   on  right  stop  \
actuals                                                                     
_silence        49         2     0    0     0    0    0    0      0     0   
_unknown         0      3030     2    2     0    1    0    0      1     1   
down             0         7   167    0     0    1    0    0      0     0   
go               0         5     0  161     0    0    0    0      1     0   
left             0         9     0    0   202    0    0    0      0     0   
no               0         1     2    1     0  175    0    0      0     0   
off              0         7     0    0     0    0  167    0      0     0   
on               0        11     0    0     0    0    1  190      0     0   
right            0         9     0    0     0    0    0    0    163     0   
stop             0         4     0    0     0    0    0    0      0   186   
up               0         3 

In [24]:
preds = [LABELS[i] for i in np.argmax(preds_proba, axis=1)]
actuals = [LABELS[i] for i in np.argmax(y_valid, axis=1)]
print('* 정확도 : %.5f' % (np.sum(np.array(actuals) == np.array(preds)) / float(len(actuals))))
preds = pd.Categorical(preds, categories=LABELS)
actuals = pd.Categorical(actuals, categories=LABELS)
print('\n -------------------------- \n')
print(pd.crosstab(actuals, preds, rownames=['actuals'], colnames=['preds']))
print('\n -------------------------- \n')
print(classification_report(actuals, preds))

* 정확도 : 0.96970

 -------------------------- 

preds     _silence  _unknown  down   go  left   no  off   on  right  stop  \
actuals                                                                     
_silence        46         2     0    0     0    0    0    1      0     0   
_unknown         2      2941    14   14     8    8    8   25      4     4   
down             0         1   171    1     0    2    0    0      0     0   
go               0         2     0  161     0    1    0    1      0     0   
left             1         1     0    0   207    0    0    1      1     0   
no               0         0     2    1     0  175    0    0      0     0   
off              0         4     0    0     0    0  171    1      0     0   
on               0         4     0    0     0    0    3  194      0     1   
right            0         4     0    0     1    0    0    0    167     0   
stop             0         1     1    1     0    0    0    0      0   188   
up               0         0 

In [28]:
preds = [LABELS[i] for i in np.argmax(preds_proba, axis=1)]
actuals = [LABELS[i] for i in np.argmax(y_valid, axis=1)]
print('* 정확도 : %.5f' % (np.sum(np.array(actuals) == np.array(preds)) / float(len(actuals))))
preds = pd.Categorical(preds, categories=LABELS)
actuals = pd.Categorical(actuals, categories=LABELS)
print('\n -------------------------- \n')
print(pd.crosstab(actuals, preds, rownames=['actuals'], colnames=['preds']))
print('\n -------------------------- \n')
print(classification_report(actuals, preds))

* 정확도 : 0.96658

 -------------------------- 

preds     _silence  _unknown  down   go  left   no  off   on  right  stop  \
actuals                                                                     
_silence        63         0     0    0     0    0    0    0      0     0   
_unknown         3      2971     7   25    11    9   19   20     10     8   
down             0         0   169    2     0    3    0    0      0     0   
go               0         1     0  161     0    2    1    1      0     1   
left             0         1     0    0   173    0    0    4      0     0   
no               0         1     0    1     0  175    0    1      0     0   
off              1         0     0    1     0    0  168    1      0     0   
on               0         1     0    3     0    0    1  160      0     0   
right            2         2     0    0     0    0    0    1    154     0   
stop             1         0     0    0     0    0    0    1      0   199   
up               0         0 

In [50]:
for i in range(100):
    t= preds_proba[i].round(decimals=2)
    print(t)

[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
[ 0.          0.95999998  0.03        0.          0.          0.          0.
  0.          0.          0.          0.          0.        ]
[ 0.          0.          0.          0.99000001  0.          0.          0.
  0.          0.          0.          0.          0.        ]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
[ 0.  1.  0.  0.  0.  

In [43]:
preds = [LABELS[i] for i in np.argmax(preds_proba, axis=1)]
actuals = [LABELS[i] for i in np.argmax(y_valid, axis=1)]
print('* 정확도 : %.5f' % (np.sum(np.array(actuals) == np.array(preds)) / float(len(actuals))))
preds = pd.Categorical(preds, categories=LABELS)
actuals = pd.Categorical(actuals, categories=LABELS)
print('\n -------------------------- \n')
print(pd.crosstab(actuals, preds, rownames=['actuals'], colnames=['preds']))
print('\n -------------------------- \n')
print(classification_report(actuals, preds))

* 정확도 : 0.98288

 -------------------------- 

preds     _silence  _unknown  down   go  left   no  off   on  right  stop  \
actuals                                                                     
_silence        63         0     0    0     0    0    0    0      0     0   
_unknown         3      3075     1    4     0    3    1    2      2     2   
down             0         2   170    1     0    0    0    0      0     0   
go               0         4     0  158     0    3    0    0      0     2   
left             0         6     0    0   171    0    0    0      0     0   
no               0         1     1    3     0  172    0    0      0     0   
off              1         2     0    0     0    0  162    2      0     0   
on               0         6     0    1     0    0    4  155      0     0   
right            2         2     0    0     1    0    0    0    153     0   
stop             1         3     0    0     0    0    0    0      0   198   
up               0         2 

In [12]:
preds = [LABELS[i] for i in np.argmax(preds_proba, axis=1)]
actuals = [LABELS[i] for i in np.argmax(y_valid, axis=1)]
print('* 정확도 : %.5f' % (np.sum(np.array(actuals) == np.array(preds)) / float(len(actuals))))
preds = pd.Categorical(preds, categories=LABELS)
actuals = pd.Categorical(actuals, categories=LABELS)
print('\n -------------------------- \n')
print(pd.crosstab(actuals, preds, rownames=['actuals'], colnames=['preds']))
print('\n -------------------------- \n')
print(classification_report(actuals, preds))

* 정확도 : 0.97493

 -------------------------- 

preds     _silence  _unknown  down   go  left   no  off   on  right  stop  \
actuals                                                                     
_silence        63         0     0    0     0    0    0    0      0     0   
_unknown         5      3014     1    7     4    6    6    9     20    12   
down             0         2   169    2     0    1    0    0      0     0   
go               0         5     0  160     0    1    0    0      0     0   
left             0         1     0    0   173    0    0    2      0     1   
no               0         0     0    2     0  175    0    0      0     1   
off              1         0     0    1     0    0  166    1      0     0   
on               0         2     1    2     0    0    1  160      0     0   
right            2         2     0    0     0    0    0    0    154     0   
stop             0         0     0    0     0    0    1    1      0   200   
up               0         0 

In [14]:
preds = [LABELS[i] for i in np.argmax(preds_proba, axis=1)]
actuals = [LABELS[i] for i in np.argmax(y_valid, axis=1)]
print('* 정확도 : %.5f' % (np.sum(np.array(actuals) == np.array(preds)) / float(len(actuals))))
preds = pd.Categorical(preds, categories=LABELS)
actuals = pd.Categorical(actuals, categories=LABELS)
print('\n -------------------------- \n')
print(pd.crosstab(actuals, preds, rownames=['actuals'], colnames=['preds']))
print('\n -------------------------- \n')
print(classification_report(actuals, preds))

* 정확도 : 0.97330

 -------------------------- 

preds     _silence  _unknown  down   go  left   no  off   on  right  stop  \
actuals                                                                     
_silence        63         0     0    0     0    0    0    0      0     0   
_unknown         9      3013     4    7     5    5    7   13     13     8   
down             0         1   171    1     0    1    0    0      0     0   
go               0         3     0  158     0    2    0    0      1     2   
left             0         1     0    0   172    1    1    2      0     0   
no               0         0     1    1     0  175    0    0      0     0   
off              1         1     0    1     0    0  166    1      0     0   
on               0         4     0    1     0    0    1  159      0     0   
right            3         1     0    0     0    0    0    0    154     0   
stop             1         1     0    0     0    0    1    1      0   197   
up               1         0 

In [26]:
preds = [LABELS[i] for i in np.argmax(preds_proba, axis=1)]
actuals = [LABELS[i] for i in np.argmax(y_valid, axis=1)]
print('* 정확도 : %.5f' % (np.sum(np.array(actuals) == np.array(preds)) / float(len(actuals))))
preds = pd.Categorical(preds, categories=LABELS)
actuals = pd.Categorical(actuals, categories=LABELS)
print('\n -------------------------- \n')
print(pd.crosstab(actuals, preds, rownames=['actuals'], colnames=['preds']))
print('\n -------------------------- \n')
print(classification_report(actuals, preds))

* 정확도 : 0.97028

 -------------------------- 

preds     _silence  _unknown  down  go  left   no  off  on  right  stop   up  \
actuals                                                                        
_silence        54         0     0   0     0    0    0   0      0     0    0   
_unknown         4      2029     0   7     1    9    2   1      0     0    4   
down             0         4   128   1     0    1    0   0      0     0    0   
go               0         5     4  87     0    2    0   0      0     0    0   
left             0         6     0   0   126    0    0   0      0     0    2   
no               0         4     2   0     0  103    0   0      0     1    0   
off              1         6     1   0     1    0  100   0      0     1    1   
on               0         4     0   0     0    0    1  98      0     0    0   
right            0         5     0   0     1    0    0   0    121     0    0   
stop             0         4     1   0     0    1    0   0      0   118  

In [ ]:
%%time
y = []
X = []
for i, (label, fname) in enumerate(zip(labels, fnames)):
    sample_rate, samples = wavfile.read(os.path.join(TRAIN_PATH, label, fname))
    if len(samples) > 16000:
        pass
    else:
        samples = pad_audio(samples)
#         resampled = signal.resample(samples, int(new_sample_rate / sample_rate * samples.shape[0]))
#         _, _, specgram = log_specgram(resampled, sample_rate=new_sample_rate)
        _, _, specgram = log_specgram(samples, sample_rate=16000)
        y.append(label)
        X.append(specgram)

In [17]:
# model = load_model(os.path.join(OUTPUT_PATH, 'cnn_baseline_again_no_cw.h5'))

# 제출파일 생성

In [62]:
%%time
X = []
submission_fpaths = sorted(glob(os.path.join(TEST_PATH, r'*wav')))
for fpath in submission_fpaths:
    sample_rate, samples = wavfile.read(fpath)
    _, _, specgram = log_specgram(samples, sample_rate=16000)
    X.append(specgram)

CPU times: user 2min 16s, sys: 3.25 s, total: 2min 19s
Wall time: 2min 17s


In [64]:
X = np.array(X)
X = X.reshape(tuple(list(X.shape) + [1]))    

In [65]:
preds_proba = model.predict(X, batch_size=2048, verbose=1)
preds = [[L.replace('_', '') for L in LABELS][i] for i in np.argmax(preds_proba, axis=1)]

158538/158538 [==============================] - 26s    


In [66]:
df = pd.DataFrame({'fname': submission_fpaths, 'label': preds})
df['fname'] = df['fname'].apply(lambda p: p.split('/')[-1])
df.to_csv(os.path.join(OUTPUT_PATH, 'sub_' + MODEL_NAME.split('.')[0] + '.csv'), index=False)


In [69]:
preds_proba.round(decimals=2)

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.08      ,  0.        , ...,  0.92000002,
         0.        ,  0.        ],
       [ 0.52999997,  0.43000001,  0.        , ...,  0.        ,
         0.        ,  0.02      ],
       [ 0.        ,  1.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

In [ ]:

for fpath in submission_fpaths:
    sample_rate, samples = wavfile.read(fpath)
    samples = pad_audio(samples)
    if len(samples) > 16000:
        n_samples = chop_audio(samples)
    else: 
        n_samples = [samples]
    for samples in n_samples:
#         resampled = signal.resample(samples, int(new_sample_rate / sample_rate * samples.shape[0]))
#         _, _, specgram = log_specgram(resampled, sample_rate=new_sample_rate)
        _, _, specgram = log_specgram(samples, sample_rate=16000)
        X.append(specgram)
        
X = np.array(X)
X = X.reshape(tuple(list(X.shape) + [1]))

In [ ]:
preds_proba = model.predict(X, batch_size=2048, verbose=1)
preds = [[L.replace('_', '') for L in LABELS][i] for i in np.argmax(preds_proba, axis=1)]

In [ ]:
df = pd.DataFrame({'fname': submission_fpaths, 'label': preds})
df['fname'] = df['fname'].apply(lambda p: p.split('/')[-1])
df.to_csv(os.path.join(OUTPUT_PATH, 'sub_' + MODEL_NAME.split('.')[0] + '.csv'), index=False)

In [50]:
# cw
df['label'].value_counts()

unknown    101125
off          6768
no           6155
on           5917
go           5738
up           5573
yes          5297
stop         5254
left         5175
down         4683
right        3983
silence      2870
Name: label, dtype: int64

In [21]:
# no cw
df['label'].value_counts()

unknown    100749
up           6000
off          5928
no           5673
silence      5619
go           5218
left         5197
stop         5191
on           5160
yes          5046
right        4558
down         4199
Name: label, dtype: int64

In [28]:
df['label'].value_counts()

unknown    83693
on         12284
go          7641
up          7575
no          7171
off         6637
yes         6463
left        6252
stop        5649
down        5579
right       5492
silence     4102
Name: label, dtype: int64

In [30]:
df['label'].value_counts()

unknown    100064
up           6329
off          6132
on           5977
no           5945
go           5595
yes          5563
stop         5346
left         5344
down         4911
right        4691
silence      2641
Name: label, dtype: int64

In [33]:
#before
df['label'].value_counts()

unknown    79111
on         17357
no          8149
off         7381
go          7010
up          6905
left        6389
yes         5975
stop        5941
right       5803
down        4760
silence     3757
Name: label, dtype: int64

In [38]:
# weight 안준거
df['label'].value_counts()

unknown    100136
off          6243
stop         5855
on           5683
left         5659
yes          5567
no           5381
go           5380
up           5305
right        5006
down         4417
silence      3906
Name: label, dtype: int64

In [27]:
# weight 준거
df['label'].value_counts()

unknown    82664
on         16551
off         7639
no          7616
left        5922
yes         5848
right       5774
go          5647
down        5622
stop        5601
up          5448
silence     4206
Name: label, dtype: int64

In [47]:
df['label'].value_counts()

unknown    100591
yes          6519
off          6272
stop         6064
left         5637
no           5428
up           5392
go           5237
down         5135
on           5126
right        4923
silence      2214
Name: label, dtype: int64

In [16]:
df['label'].value_counts()

unknown    81043
on         14810
off         7550
up          7044
right       6950
left        6659
go          6553
stop        6360
yes         6292
no          6199
down        4760
silence     4318
Name: label, dtype: int64

In [18]:
df['label'].value_counts()

unknown    82108
on         15084
off         8341
no          6978
right       6541
left        6510
up          6284
yes         6137
stop        5775
go          5678
down        4916
silence     4186
Name: label, dtype: int64